### import Libraries

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer

/Users/takahashikaisei/mydir/transforemer_from_scratch/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


おまじない．

In [2]:
# displayしたときにshape, numel, dtypeがわかるようにする．神
def custom_repr(self):
    return f"{tuple(self.shape)}[{self.numel()}]: {str(self.dtype).lstrip('torch.')} = {original_repr(self)}"


original_repr = torch.Tensor.__repr__
torch.Tensor.__repr__ = custom_repr

# Transformer

例のコレ

<img src="./image/transformer.png" width="400px" alt="transformer">

ここではテキスト生成を目的とするので，decoder部分のみのtransformerを実装する．

decoder onlyのtransformerのアーキテクチャはコレ．なんかググったら出てきた．

<img src="./image/transformer_decoder.png" width="200px" alt="transformer_decoder">

## trasformer layerに入れる前の準備

アーキテクチャをみると，テキストを入力してからembedding, positional encodingをしてからtransformer layerに入れている．

その工程を一個一個実装していく．

### tokenize

まずはテキストをtoken化する．数字じゃないと使えないので．

ということでトークナイザーを準備する，

また，id化したものをtensorに変換しておく．

In [3]:
# ここではgpt2tokenizerを使う．これが手っ取り早い．
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
sample_text = "私はAIです。"
input_ids = torch.tensor(gpt2_tokenizer.encode(sample_text))
display(input_ids)

(8,)[8]: int64 = tensor([  163,   100,   223, 31676, 20185, 30640, 33623, 16764])

id化できた．length=8だ．

idは元の字に対応しているのでその対応関係を見てみる．

decodeすればいいだけ．

In [4]:
display(gpt2_tokenizer.decode([163, 100, 223]))
display(gpt2_tokenizer.decode([31676]))
display(gpt2_tokenizer.decode([20185]))
display(gpt2_tokenizer.decode([30640]))
display(gpt2_tokenizer.decode([33623]))
display(gpt2_tokenizer.decode([16764]))

'私'

'は'

'AI'

'で'

'す'

'。'

謎に"私"で3つidが割り当てられている．これはなんでだろう？

それは，各idは以下のように対応しているのが答え．

私にidが3つ振られているのは，unicodeの組み合わせで表現されているから．

<img src="./image/tokenize_example.png" width="400px" alt="tokenize_example">

unicodeと文字が一対一対応ならレンダリングされるけど，組み合わせでできてる"私"は潰れてしまってる．

### embedding

tokenizeできたら，次はembeddingをする．

tokenを一個一個ベクトルにして，計算処理ができるようにする．

ちなみにここでできるベクトルは最初は特に意味がないランダムなベクトル．

In [5]:
vocab_size = (
    gpt2_tokenizer.vocab_size
)  # tokenizerの語彙数．出力時には，これを確率分布で表現する．
display(vocab_size)  #
d_model = 512  # 埋め込み次元数，attention is all you needに合わせる．
embed = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)

50257

この埋め込みベクトルは学習によって言語の分散表現を獲得する．

したがってparameters()で取得できる．

だから学習可能パラメータを定義していると考えればいいね．

In [6]:
embed

Embedding(50257, 512)

In [7]:
display(list(embed.parameters()))

[Parameter containing:
 (50257, 512)[25731584]: float32 = tensor([[ 1.5396,  0.0155,  1.2735,  ..., -0.7754, -0.0243,  0.1280],
         [-0.1923,  0.0127, -0.4817,  ...,  0.3117,  1.7169,  0.8467],
         [-0.1210, -1.0940, -1.6180,  ...,  0.0884,  0.4523,  0.6500],
         ...,
         [ 0.0510, -0.9768,  0.6745,  ...,  0.2911,  1.4849,  0.9392],
         [ 1.6317,  1.3134, -0.2387,  ..., -0.3693, -0.5047, -0.0954],
         [ 0.8078,  1.4899,  0.4799,  ...,  0.5571, -1.6068,  0.1679]],
        requires_grad=True)]

In [8]:
embed_input = embed(input_ids)
embed_input.size()

torch.Size([8, 512])

もちろん，8個のidを振られた文章をembeddingしているので，idを取得すると(8, 512)となっている．

### positional encoding

Attention is all you need 論文では，位置情報を加えるためにsine/cosine関数を用いた位置エンコーディングを使っている．

しかし，ここでは簡単のために，単純に学習可能なパラメータとして位置エンコーディングを実装する．

positional encodingは，単純にembeddingされたベクトルに足し合わせるだけ．

位置ごとに割り振られた学習可能パラメータをただ足すだけで良い．

In [9]:
embed_input.size()

torch.Size([8, 512])

上記のembed_inputがpostional encodingをする前のベクトル．これに位置情報を足す．

なのでsizeは変わらない．

In [18]:
context_window = 100  # コンテキストウィンドウ．tokenいくつまでを処理するか．プロプライエタリモデルのドキュメントを見ると，絶対かいてある．これ以上はカットされる．
# ここでは100tokenしか処理しない．本当はもっと大きいよ．
d_model = embed_input.size()[1]

In [20]:
pe = nn.Parameter(torch.randn([context_window, d_model]))
display(pe)

Parameter containing:
(100, 512)[51200]: float32 = tensor([[-1.4986, -2.6413,  0.4043,  ..., -0.3874,  0.9318,  0.1330],
        [ 0.7439,  0.3280,  1.0592,  ..., -0.3740,  0.6549,  0.8521],
        [ 0.5973, -0.2180,  0.2851,  ..., -1.6767, -0.5493,  0.3618],
        ...,
        [ 1.3827,  0.9211,  1.6553,  ..., -0.6416, -1.6116, -1.2377],
        [ 0.6457, -1.5711, -0.6449,  ..., -0.5333,  2.4517, -2.6644],
        [ 1.1070, -0.7889,  0.2765,  ...,  1.7800,  0.9920,  0.7272]],
       requires_grad=True)

(100, 50257)のベクトルができた．

単純にスライスして，必要な長さだけをembed_inputに足し合わせれば良い．

In [22]:
embed_input + pe[embed_input.size()[0], :]

(8, 512)[4096]: float32 = tensor([[ 1.4865, -2.0641, -2.9895,  ..., -0.4920,  2.0101, -2.2965],
        [ 0.7928, -1.0908, -0.6428,  ...,  1.1935,  0.4169, -1.3922],
        [-0.1188, -3.1827, -0.9049,  ...,  0.7935, -0.6548, -2.3558],
        ...,
        [-1.4752, -0.3560,  0.2262,  ...,  1.8922,  0.6387, -1.4002],
        [-1.1464, -0.4868, -1.3613,  ...,  0.6582,  2.6510, -1.4161],
        [ 1.8512,  0.1382, -1.3188,  ...,  1.2771, -0.5142, -1.2817]],
       grad_fn=<AddBackward0>)

これでtransformer layerに入れる準備ができた．

## transformer layer

### Scaled Dot-Product Attention

いよいよtransformer layerを実装する．

まずscaled dot-product attentionを実装することから始める．

やることはいたってシンプル．

<img src="./image/scaled_dot_product_attention.png" width="400px" alt="scaled_dot_product_attention.png">

もとい，

$$
\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

をそのまま書くだけ．

数式を実装するにあたって，どうすればいいかというと，右辺の式でしか登場しない変数はコンストラクタで定義して，左辺で渡されている変数はforwardの引数にすればいい．

これがコツ．

In [ ]:
class Attention(nn.Module):
    def __init__(self, d: int):
        super().__init__()
        self.scale = d ** (
            1 / 2
        )  # 右辺で新たに渡している要素はdのみ．コンストラクタで定義する．
        self.softmax = nn.Softmax(
            dim=-1
        )  # batchが入ってきたときに対応できるよう，-1にしておく．softmaxはkey方向に取りたい．それは横方向．
        # ここが，dim=1だとbatch次元が増えたときに，一個ずれてしまう．不本意に縦方向をとってしまう．

    # 左辺をそのまま書く．左辺で新たに渡している要素はQ, K, Vだよね．
    def forward(self, q, k, v):
        """
        Args:
            q: seq_length × embedding_size(8×512)
            k: seq_length × embedding_size(8×512)
            v: seq_length × embedding_size(8×512)
        """
        score = (q @ k.T) / self.scale  # 8×8
        return self.softmax(score) @ v  # 8×8 @ 8×512 = 8×512

双方向のattentionが実装できた．

しかし，双方向のattentionはgpt系のモデルでは使わない．いわゆるcausal attentionを実装する必要がある．

なにをするかというと，未来の情報を参照しないようにmaskをかける．

その前にbatch化したときに問題なく実装できるように，tensorの挙動を確認しておこう．

In [23]:
x = torch.rand(2, 8, 512)  # (batch, seq_len, d_model)を想定
display(x.size())

torch.Size([2, 8, 512])

これを転置したい．

今までは`.T`で良かったけど，batch軸が足されたtensorの場合どうだろうか？

In [26]:
display(x.T.size())

torch.Size([512, 8, 2])

batch軸, d_model軸で入れ替えられた．

これは不本意．本当はseq_lenとd_modelの軸を入れ替えたい．

それは`.mT`をやるとうまくいく．mTはおしりの二軸を入れ替える操作．

In [27]:
display(x.mT.shape)

torch.Size([2, 512, 8])

現状の双方向のattentionだと，queryが渡された時に未来の情報がリークしている．

このままだと，未来の情報を参照してしまうので，次単語予測がうまく学習できない．

どういうことか．

"私はAIです。"という文章があり，この文章がそのままquery, keyに対応するとする．

この時,query, keyのマトリクスは以下．
| query\key  | 私 | は | A | I | です | 。 |
|---|---|---|---|---|---|---|
| 私 ||||||
| は ||||||
| A ||||||
| I ||||||
| です ||||||
| 。 ||||||

双方向の例は以下．

| query\key  | 私 | は | A | I | です | 。 |
|---|---|---|---|---|---|---|
| 私 ||||||

このようにqueryに"私"が来ているのに，keyでそれ以降の未来の単語情報を与えてしまうのはリークしている．

この状態だとまともに学習もできない．

単方向にする必要がある．

単方向にするにはmaskをかければ良い．

| query\key  | 私 | は | A | I | です | 。 |
|---|---|---|---|---|---|---|
| 私 ||///|///|///|///|///
| は |||///|///|///|///
| A ||||///|///|///
| I |||||///|///
| です ||||||///
| 。 ||||||

この///の部分はattentionをとらない．

そうすれば未来の情報を参照しなくなる．

これがmaskのイメージ．

実装する．

In [34]:
# 全ての要素が1の行列を作って，上三角行列を飛ばしてboolにする．Trueの場所を-infにするのがmask
seq_len = 8
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
display(mask.shape)
print(f"{mask=}")

torch.Size([8, 8])

mask=(8, 8)[64]: bool = tensor([[False,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False]])


で，バッチ軸を足したときに，このtensorはうまくブロードキャストしてくれるのかが気になる．

→してくれる．

In [39]:
# こういうのが欲しい．
seq_len = 8
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
display(mask.shape)
unsqueezed = mask.unsqueeze(0)  # batch軸を足す操作．indexを指定．
display(unsqueezed.shape)

torch.Size([8, 8])

torch.Size([1, 8, 8])

In [40]:
unsqueezed

(1, 8, 8)[64]: bool = tensor([[[False,  True,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True,  True],
         [False, False, False, False,  True,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False, False,  True,  True],
         [False, False, False, False, False, False, False,  True],
         [False, False, False, False, False, False, False, False]]])

causal attentionを作る．

単方向のattentionはcausal attentionと呼ばれる．

まあ式は一緒．


$$
\text{Attention}(Q, K, V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$


In [41]:
class CausalAttention(nn.Module):
    def __init__(self, d: int):
        super().__init__()
        self.scale = d ** (1 / 2)  # 右辺で新たに渡している要素はdのみ
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v):
        """
        Args:
            q: (seq_length, embedding_size)
            k: (seq_length, embedding_size)
            v: (seq_length, embedding_size)
        """
        seq_len = q.shape[-2]
        score = (q @ k.mT) / self.scale
        # 増えたところ↓
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
        score = score.masked_fill(mask, -torch.inf)  # Trueを-infに飛ばす．

        return self.softmax(score) @ v

attentionのイメージを完璧に掴むため，自力で計算してみよう．

In [54]:
batch_size = 2
seq_len = 3
d_model = 4

q = torch.randn(batch_size, seq_len, d_model)
k = torch.randn(batch_size, seq_len, d_model)
v = torch.randn(batch_size, seq_len, d_model)

q = torch.tensor([[1, 0], [1, 1], [2, 2]]).unsqueeze(0).to(torch.float32)
k = torch.tensor([[1, 0], [1, 1], [2, 2]]).unsqueeze(0).to(torch.float32)
v = torch.tensor([[5, 0], [5, 5], [-3, 100]]).unsqueeze(0).to(torch.float32)

以下のような値がq, k, vであったとき，どういうoutputになるか．

計算の簡単のために，`scale = 1`で, softmaxではなく，重みが均等な確率分布の処理を考える．

一単語ごとに内積を計算する．

- query一個目[1, 0]の処理

`q = [1, 0]`がのとき，対応するkは`k = [1, 0]`

$$
q\cdot k^T = 1 \\

1\cdot v = 1\cdot [5, 0] = [5, 0]
$$

- query二個目[1, 1]の処理

`q = [1, 1]`がのとき，対応するkは`k = [1, 0], [1, 1]`

$$
\begin{align*}
q\cdot k^T &= \begin{bmatrix} 1 , 1 \end{bmatrix} \cdot \begin{bmatrix} 1 , 1 \\ 0, 1 \end{bmatrix}\\
&= \begin{bmatrix} 1 , 2 \end{bmatrix}\\
\end{align*}
$$
確率分布にする．
$$
\begin{bmatrix} 0.33, 0.66 \end{bmatrix}
$$

$$
\begin{bmatrix} 0.33, 0.66 \end{bmatrix} \times v = 0.33\cdot [5, 0] + 0.66\cdot [5, 5] = [5.0, 3.3]
$$

- query三個目[2, 2]の処理

`q = [2, 2]`がのとき，対応するkは`k = [1, 0], [1, 1], [2, 2]`

$$
\begin{align*}
q\cdot k^T &= \begin{bmatrix} 2 , 2 \end{bmatrix} \cdot \begin{bmatrix} 1 , 0 \\ 1, 1 \\ 2, 2 \end{bmatrix}\\
q\cdot k^T &= \begin{bmatrix} 2 , 2 \end{bmatrix} \cdot \begin{bmatrix} 1, 1, 2 \\ 0, 1, 2 \end{bmatrix}\\
&= \begin{bmatrix} 2 , 4, 8 \end{bmatrix}\\
\end{align*}
$$
確率分布にする．
$$
    \begin{bmatrix} 0.14, 0.28, 0.57 \end{bmatrix}
$$

$$
    \begin{bmatrix} 0.14, 0.28, 0.57 \end{bmatrix} \times v = 0.14\cdot [5, 0] + 0.28\cdot [5, 5] +  0.57\cdot [-3, 100]= [0.39, 58.4]
$$

まとめると...

$$
\text{CausalAttention}(Q, K, V) = \begin{bmatrix} 5.0, 0.0 \\ 5.0, 3.3 \\ 0.39, 58.4 \end{bmatrix}
$$


In [57]:
attention = CausalAttention(1)
y = attention(q, k, v)
y

(1, 3, 2)[6]: float32 = tensor([[[ 5.0000,  0.0000],
         [ 5.0000,  3.6553],
         [-2.8370, 98.0526]]])

実際の答えとはズレた，コレはsoftmaxが原因．

query3個目の確率分布は[0.14, 0.28, 0.57]ではなく，softmaxで計算すると[0.00, 0.01, 0.97]になる．

ここで結構ズレるので最終的な結果も大幅にズレた．

でもAttentionの気持ちはコレで掴めた．

query一個に着目して，それとkeyの内積をとる．内積の値がそのままscore(単語同士の重要度みたいなもん)で，それを確率分布にする．

確率分布でvalueのベクトルごとの重要度を操作．

最後に和をとる．(いわゆる重み付き和を計算することになっている．)

ちなみにtorchで実装されてるやつと実装はちゃんと合ってる．

In [58]:
from torch.nn import functional as F

F.scaled_dot_product_attention(q, k, v, is_causal=True, scale=1)

(1, 3, 2)[6]: float32 = tensor([[[ 5.0000,  0.0000],
         [ 5.0000,  3.6553],
         [-2.8370, 98.0526]]])

ポイントは時刻tごとの演算をすることなのです．

### Multi Head Attention

コレ．

<img src="./image/mha.png" width="300px" alt="mha.png">

数式は以下．

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_{\text{h}})W^O\\
\text{where}\text{ } \text{head}_{\text{i}} = \text{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

これも簡単で，埋め込みベクトルをhead数で分割して，それごとにAttentionを計算．

headに分割したのを元に戻して線形層を噛ませる．

てかそもそもQ, K, Vってどうやって作るねんって話だよね．

以下の図をみればわかる．

<img src="./image/transformer_decoder.png" width="200px" alt="transformer_decoder">

一個の入力から分岐してる．

つまり入力の(batch_size, seq_len, d_model)のベクトルをそのまま使ってんだね．

これを線形層に通して，Attentionに突っ込む．

$QW^Q_i, KW^K_i, VW^V_i$って書いてあって，パラメータの行列がかかってるけど，意味は線形層と全く一緒．

なので`torch.nn.Linear`で実装すれば良い．

アーキテクチャ的には，headに分割してから線形層に通してるけど，線形層に通してからheadに分割しても同じなので，後者の方が実装は楽．それで実装する．

In [59]:
class MultiHeadAttention(nn.Module):
    def __init__(
        self, n_head, d_model
    ):  # ここは64じゃなくてn_headの方が良い．契約プログラミング的に．64とすると全体で見た時マジックナンバーっぽくなる．
        super().__init__()
        assert d_model % n_head == 0, "割り切れないよ"
        self.n_head = n_head
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)
        self.attention = CausalAttention(d_model)

    def forward(self, q, k, v):
        qw = self.w_q(q)
        kw = self.w_k(k)
        vw = self.w_v(v)
        calculated = []
        # chunkでhead数に分割する．
        for qw_i, kw_i, vw_i in zip(
            qw.chunk(self.n_head, dim=-1),
            kw.chunk(self.n_head, dim=-1),
            vw.chunk(self.n_head, dim=-1),
        ):
            calculated.append(self.attention(qw_i, kw_i, vw_i))
        return self.w_o(torch.cat(calculated, dim=-1))

In [72]:
n_head = 8
multi_head_attention = MultiHeadAttention(n_head, 256)
n_param = sum(
    p.numel() for p in multi_head_attention.parameters() if p.requires_grad
)
print(f"{n_param=}")

n_param=263168


In [74]:
batch_size = 1
seq_len = 10
d_model = 256

q = torch.randn(batch_size, seq_len, d_model)
k = torch.randn(batch_size, seq_len, d_model)
v = torch.randn(batch_size, seq_len, d_model)
multi_head_attention(q, k, v)
n_param = sum(
    p.numel() for p in multi_head_attention.parameters() if p.requires_grad
)
print(f"{n_param=}")

n_param=263168


In [71]:
mha = nn.MultiheadAttention(d_model, n_head)
mha(q, k, v)

n_param = sum(p.numel() for p in mha.parameters() if p.requires_grad)
print(f"パラメータ数: {n_param}")

パラメータ数: 263168


できた．

### Feed Forward Network

multi head attentionの後はFeed Forward Network(FFN)を通す．

ここに記憶が詰まるとか言われてんだっけね．

数式は以下．

$$
\text{FFN}(x) = \text{max}(0, xW_1 + b_1)W_2 + b_2
$$

この数式の意味は線形層に通して，その後にReLUを通して，さらに線形層に通してるだけ．

隠れ層の次元数はd_ffとして記述されていて，論文中では2048に設定されている．

In [75]:
class FFN(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff), nn.ReLU(), nn.Linear(d_ff, d_model)
        )

    def forward(self, x):
        return self.net(x)

In [78]:
ffn = FFN(2, 8)
ffn(torch.rand(3, 7, 2))  # batch_size, seq_len, d_model
# batch_firstのオプションもある．

(3, 7, 2)[42]: float32 = tensor([[[ 0.2097, -0.0288],
         [ 0.1410, -0.0016],
         [ 0.1480, -0.1233],
         [ 0.1540,  0.0016],
         [ 0.2656, -0.0517],
         [ 0.1744, -0.0264],
         [ 0.2467, -0.0439]],

        [[ 0.1932, -0.0204],
         [ 0.1925, -0.0477],
         [ 0.1425, -0.0987],
         [ 0.1447, -0.0189],
         [ 0.1603, -0.0509],
         [ 0.2292, -0.0801],
         [ 0.2369, -0.0392]],

        [[ 0.2678, -0.0530],
         [ 0.1079, -0.1322],
         [ 0.2106, -0.0277],
         [ 0.1432,  0.0058],
         [ 0.1351, -0.0789],
         [ 0.2769, -0.0792],
         [ 0.1987, -0.1195]]], grad_fn=<ViewBackward0>)

### 合わせる

これでtransformer layerの全ての構成要素が揃った．

これらを組み合わせる．

素直にそのまま書くだけ．

インスタンス変数には，今まで出てきたインスタンス変数を持たせる．

また，residual connectionとpost layer normalizationも忘れずに入れる．

In [79]:
class TransformerLayer(nn.Module):
    def __init__(self, n_head, d_model, d_ff):
        super().__init__()
        self.mha = MultiHeadAttention(n_head, d_model)
        self.ffn = FFN(d_model, d_ff)
        self.layer_norm_1 = nn.LayerNorm(
            d_model
        )  # normレイヤーはパラメータを共有しているわけでは無いので，二つインスタンスが必要．
        self.layer_norm_2 = nn.LayerNorm(d_model)

    def forward(self, x):  # (batch, seq_len, d_model)
        residual_1 = x  # 接続用に保存しておく．
        x = self.mha(x)
        x = x + residual_1  # 残差結合
        x = self.layer_norm_1(x)  # post norm
        residual_2 = x
        x = self.ffn(x)
        x = x + residual_2
        x = self.layer_norm_2(x)
        return x

In [115]:
from torchviz import make_dot

In [125]:
model = TransformerLayer(8, 512, 2048)
y = model(torch.rand(4, 8, 512))

In [118]:
image = make_dot(y, params=dict(model.named_parameters()))
image.format = "png"
image.render("TransformerLayer")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'TransformerLayer.png'

## TransformerClass

transformerlayerを積み重ねて，TransformerClassを作る．

In [ ]:
class Transformer(nn.Module):

    def __init__(
        self, vocab_size, n_head, d_model, d_ff, n_layers, context_window
    ):
        super().__init__()
        self.embed = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=d_model
        )
        self.transformer_layers = nn.Sequential(
            *[TransformerLayer(n_head, d_model, d_ff) for _ in range(n_layers)]
        )  # いけてる？
        self.linear = nn.Linear(
            d_model, vocab_size
        )  # batch_size, seq_len, vocab_sizeになる
        self.softmax = nn.Softmax(dim=-1)
        self.pe = nn.Parameter(
            torch.randn(
                [context_window, d_model]
            )  # ここvocab_sizeじゃなくてd_modelじゃね？
        )  # 上限までベクトルを作って，forwardで削れば良い．

    def forward(self, x):  # batch_size, seq_len: int
        x = self.embed(x)  # batch_size, seq_len, d_model
        x = x + self.pe[:, x.size(-2), :]
        x = self.transformer_layers(x)
        x = self.linear(x)
        return self.softmax(x)

In [81]:
vocab_size = gpt2_tokenizer.vocab_size
n_head = 8
d_model = 512
d_ff = 2048
n_layers = 6
context_window = 1024

In [ ]:
transformer = Transformer(
    vocab_size, n_head, d_model, d_ff, n_layers, context_window
)

# transformer
n_param = sum(p.numel() for p in transformer.parameters() if p.requires_grad)
print(f"{n_param=:,}")
# 1億2000万のパラメータ数！！！
# float32の場合，1パラメータ4バイト換算される．
model_size = n_param * 4
print(f"{model_size=:,}")

n_param=70,952,017
model_size=283,808,068


In [83]:
true_transformer = nn.TransformerEncoder(
    nn.TransformerEncoderLayer(
        d_model, n_head, d_ff, batch_first=True, dropout=0.0
    ),
    num_layers=n_layers,
)
n_param = sum(
    p.numel() for p in true_transformer.parameters() if p.requires_grad
)
print(f"{n_param=:,}")
# embedding, pe, fcを除いた場合と同じパラメータ数！！！

n_param=18,914,304


input: `(batch_size, seq_len)` ※ int (idになってる)

transformerlayer: `(batch_size, seq_len, d_model)`

output: `(batch_size, seq_len, vocab_size)` ※ 次vocab次元のベクトルになっていればOK


例としては，

”今日もいい天気ですね”がtoken化され，”今日", "も", ”いい”, "天気", "です"，”ね”になり，これらが入力されるとしたら，transformerの出力は，

- ”今日”の次の単語の確率分布
-   ”今日も”の次の単語の確率分布
-   ”今日もいい”の次の単語の確率分布
-   ”今日もいい天気”の次の単語の確率分布
-   ”今日もいい天気です”の次の単語の確率分布
-   ”今日もいい天気ですね”の次の単語の確率分布

ということになる．

この確率分布は，vocab_size次元のベクトルで表現される．

確認する．

transformerを一つの線形層で表現するなら以下のようになる．

In [95]:
batch_size, seq_len, d_model = 4, 3, 5
vocab_size = 7
sample = torch.rand([batch_size, seq_len, d_model])
linear = nn.Linear(d_model, vocab_size)  # shapeの最後の次元が変換される．-1軸
output = linear(sample)
output.reshape(batch_size, seq_len, vocab_size)

# seq_len, d_model => seq_len, vocab_size

(4, 3, 7)[84]: float32 = tensor([[[ 0.1921,  0.0530,  0.0252,  0.4259,  0.4248, -0.3778,  0.5878],
         [ 0.0192,  0.0229,  0.1247,  0.2860,  0.1657, -0.2833,  0.6537],
         [ 0.1208,  0.0741,  0.2268, -0.0341,  0.2610, -0.0129,  0.3781]],

        [[ 0.1997,  0.0543,  0.2180,  0.0420,  0.4981,  0.0539,  0.3069],
         [ 0.1565, -0.0918,  0.3026,  0.3859,  0.4636, -0.3343,  0.4118],
         [-0.0115, -0.0487,  0.2160,  0.2810,  0.1767, -0.1957,  0.6262]],

        [[ 0.1670, -0.2004,  0.5206,  0.2835,  0.6557, -0.0684,  0.1956],
         [ 0.1638,  0.1049,  0.0354,  0.2540,  0.3967, -0.2199,  0.5378],
         [ 0.1319, -0.2266,  0.4141,  0.2749,  0.2086, -0.1107,  0.4553]],

        [[ 0.2336, -0.0559,  0.3827,  0.2375,  0.6575, -0.2093,  0.1862],
         [ 0.1759, -0.1723,  0.4199,  0.4498,  0.5762, -0.3621,  0.3189],
         [ 0.2337, -0.0123,  0.3160,  0.0439,  0.4091, -0.0409,  0.2558]]],
       grad_fn=<ViewBackward0>)

数式的に...
$$
input: x_1, x_2, ... x_n\\
$$
$$
model: p(x_t|x_{(t-1)}, x_{(t-2)},...,x_{(1)}) \text{ }※ p(x_t|x_{<t}) これがほしい
$$

inputに対するモデルの対数尤度
p(x_t|x_<t)に全てのxを入れる場合の対数尤度

$$
L = \log\prod_{t=1}^n p(x_t|x_{<t})

= \sum_{t=1}^n\log p(x_t|x_{<t})
$$

input x_1, x_2, x_3, x_4: int

output (確率分布が出力される．これがマスクをかけた意味！)
- p(x_1|BOS)
- p(x_2|x_1)
- p(x_3|x_1, x_2)
- p(x_4|x_1, x_2, x_3)

output shape
(batch_size, seq_len, vocab_size)

---

# 雑

参考： https://github.com/misya11p/language-models/tree/main?tab=readme-ov-file

- [小話: 機械学習系の論文実装で気をつけて読むべきほぼ唯一の箇所, 行列演算](https://note.com/cute_orchid39/n/n68e181a041fb)

In [ ]:
vocab = {
    "今日": {"は": 1},
    "カレー": {"は": 1},
    "天気": {"は": 0.5, "。": 0.5},
    "おいしい": {"。": 0.5, "カレー": 0.5},
    "は": {"今日": 0.25, "カレー": 0.25, "おいしい": 0.25, "いい": 0.25},
    "いい": {"天気": 0.5, "。": 0.5},
}

https://www.youtube.com/watch?v=mMa2PmYJlCo

https://bbycroft.net/llm

In [ ]:
data = [
    "今日 は いい 天気 です 。",
    "今日 は カレー を 食べ ました 。",
    "私 は 今日 カレー を 食べ ました 。",
    "私 は カレー が 好き です 。",
]

vocab = {}
for sent in data:  # 文章ごとに分ける
    sent = sent.split()  # 単語ごとに分ける
    for w1, w2 in zip(
        sent[:-1], sent[1:]
    ):  # w1は最後の単語を除いたもの，w2は最初の単語を除いたもの
        # 一個ずつずらしてペアにする．
        if w1 not in vocab:
            vocab[w1] = {}
        if w2 not in vocab[w1]:
            vocab[w1][w2] = 0
        vocab[w1][w2] += 1
    # このfor文では，w1を探して，vocabに入ってなかったら，空の辞書を入れる．
    # さらに，w2を探して，vocab[w1]にw2がなかったら，0を入れる．
    # 最後に，w1かつつw2のkeyの組み合わせが出てきたら，1を足す．
    # なのでここでは頻度を出してるだけ．

# 確率分布にスケールを揃える．
# 辞書にアクセスしてvalues()を持ってくれば，値のみ取り出せる．それをsumればいいだけ．
for w1 in vocab:
    total = sum(vocab[w1].values())
    for w2 in vocab[w1]:
        vocab[w1][w2] /= total

vocab

{'今日': {'は': 0.6666666666666666, 'カレー': 0.3333333333333333},
 'は': {'いい': 0.25, 'カレー': 0.5, '今日': 0.25},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 'です': {'。': 1.0},
 'カレー': {'を': 0.6666666666666666, 'が': 0.3333333333333333},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'ました': {'。': 1.0},
 '私': {'は': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

In [30]:
import markovify

In [ ]:
data = [
    "今日 は いい 天気 です 。",
    "今日 は カレー を 食べ ました 。",
    "私 は 今日 カレー を 食べ ました 。",
    "私 は カレー が 好き です 。",
]
display(data)
model = markovify.Text(data, state_size=1)  # 学習
sentence = model.make_sentence()
print(sentence)

# Noneで終わるケースはどういうケース？句読点に行きつかなかったってこと？

['今日 は いい 天気 です 。',
 '今日 は カレー を 食べ ました 。',
 '私 は 今日 カレー を 食べ ました 。',
 '私 は カレー が 好き です 。']

私 は 今日 は いい 天気 です 。


In [65]:
from torch import nn
from torch.nn import functional as F

In [ ]:
class LanguageModel(nn.Module):  # nn.Moduleを継承
    # 語彙数, 埋め込みサイズ
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()  # 親クラスの初期化
        self.embedding = nn.Embedding(n_vocab, hidden_size)  # 埋め込み
        self.fc = nn.Linear(
            hidden_size, n_vocab
        )  # 埋め込みから語彙数への線形変換

    def forward(self, x):
        h = self.embedding(x)  # (batch_size, hidden_size)
        y = self.fc(h)  # (batch_size, n_vocab)
        return y


# これの出力ってなんなの？埋め込んだものをもう一回線形変換してるだけじゃん．

In [ ]:
embed_dim = 512
model = LanguageModel(n_vocab, embed_dim)

NameError: name 'sp' is not defined

In [ ]:
64000000 ** (1 / 2)

8000.0

In [ ]:
8200000 / 512 / 2

8007.8125

In [ ]:
# 言語モデルの学習
loss_fn = nn.CrossEntropyLoss()  # 損失関数


# モデルを評価する時．
def eval_model(model):
    model.eval()  # 評価モード
    losses = []  # 損失をためておく
    with torch.no_grad():  # 勾配を計算しない．
        for x, t in test_loader:  # テストデータを，ミニバッチごとに取り出す．
            x = x.to(device)  # 訓練データ
            t = t.to(device)  # 正解
            y = model(x)  # 出力
            loss = loss_fn(y, t)  # 損失をとる．
            losses.append(
                loss.item()
            )  # item()でスカラーに変換してからappendする．
    loss = sum(losses) / len(losses)  # ミニバッチごとの損失の平均をとる．
    ppl = torch.exp(torch.tensor(loss)).item()  # perplexityに変換．
    return ppl


def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(
        n_iter=len(train_loader),
        n_epochs=n_epochs,
        unit=prog_unit,
        label="ppl train",
        agg_fn=lambda s, w: math.exp(s / w),  # ppl
    )
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            optimizer.zero_grad()  # 勾配初期化
            x = x.to(device)  # 入力
            t = t.to(device)  # 正解
            y = model(x)  # 出力
            loss = loss_fn(y, t)  # 損失計算
            loss.backward()  # 逆伝播
            optimizer.step()  # パラメータ更新
            prog.update(loss.item())  # 進捗バー更新

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [74]:
import torch

In [ ]:
# torch.stackの挙動を確認する．
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
c = torch.tensor([7, 8, 9])
d = torch.stack([a, b, c])
print(d)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h):
        """
        x: (seq_len, batch_size, input_size)
        h: (batch_size, hidden_size)
        """
        hs = []
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs)  # (seq_len, batch_size, hidden_size)
        return hs

内積を解き明かす．

query, key, valueを使用した，scaled dot-product attentionについて，その意味を解き明かしたい．

cousal attention

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, d: int):
        super().__init__()
        # self.scale = d ** (1/2) # 右辺で新たに渡している要素はdのみ
        self.scale = 1
        self.softmax = nn.Softmax(
            dim=-1
        )  # batchが入ってきたときに対応できるよう，-1にしておく．softmaxはkey方向に取りたいので，それは横方向．
        # ここが，dim=1だとbatch次元が増えたときに，一個ずれてしまう．不本意に縦をとってしまう．

    # 左辺をそのまま書く．左辺で新たに渡している要素はQ, K, Vだよね．
    def forward(self, q, k, v):
        """
        Args:
            q: seq_length × embedding_size(8×512)
            k: seq_length × embedding_size(8×512)
            v: seq_length × embedding_size(8×512)
        """
        seq_len = q.shape[-2]

        score = (q @ k.mT) / self.scale  # 8×8

        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
        score = score.masked_fill(mask, -torch.inf)

        return self.softmax(score) @ v  # 8×8 @ 8×512 = 8×512